In [1]:
import socket
import struct
import zlib
import tempfile
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from sklearn.metrics import classification_report
from tensorflow.keras.saving import register_keras_serializable
from tensorflow.keras import layers, initializers, backend as K
from sklearn.metrics import confusion_matrix, accuracy_score
import base64
import hashlib

In [2]:
# @tf.keras.saving.register_keras_serializable(package="Custom", name="binary_crossentropy_loss")
def binary_crossentropy_loss(y_true, y_pred):
    return tf.keras.losses.binary_crossentropy(y_true, y_pred)

In [3]:
# Custom Capsule Network Components
@register_keras_serializable(package="Custom")
class Length(layers.Layer):
    def call(self, inputs, **kwargs):
        return K.sqrt(K.sum(K.square(inputs), -1) + K.epsilon())
    
    def compute_output_shape(self, input_shape):
        return input_shape[:-1]
    
    def get_config(self):
        return super(Length, self).get_config()

@tf.keras.saving.register_keras_serializable(package="Custom")
class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, **kwargs):
        super().__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings

    def build(self, input_shape):
        self.input_num_capsule = input_shape[1]
        self.input_dim_capsule = input_shape[2]
        
        self.W = self.add_weight(
            shape=[1, self.input_num_capsule, self.num_capsule, self.dim_capsule, self.input_dim_capsule],
            initializer=initializers.glorot_uniform(),
            name='W'
        )
        self.built = True

    def call(self, inputs):
        inputs_expand = K.expand_dims(K.expand_dims(inputs, 2), 2)
        W_tiled = K.tile(self.W, [K.shape(inputs)[0], 1, 1, 1, 1])
        inputs_hat = tf.squeeze(tf.matmul(W_tiled, inputs_expand, transpose_b=True), axis=-1)
        b = tf.zeros(shape=[K.shape(inputs)[0], self.input_num_capsule, self.num_capsule])

        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=2)
            c_expand = K.expand_dims(c, -1)
            outputs = self.squash(tf.reduce_sum(inputs_hat * c_expand, axis=1))
            if i < self.routings - 1:
                b += tf.reduce_sum(inputs_hat * K.expand_dims(c, -1), axis=-1)
        
        return outputs
    def get_config(self):
        config = super().get_config()
        config.update({
            "num_capsule": self.num_capsule,
            "dim_capsule": self.dim_capsule,
            "routings": self.routings
        })
        return config
    def squash(self, vectors, axis=-1):
        s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
        scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
        return scale * vectors

@tf.keras.saving.register_keras_serializable(package="Custom", name="margin_loss")
def margin_loss(y_true, y_pred):
    y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=2)
    L = y_true * tf.square(tf.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * tf.square(tf.maximum(0., y_pred - 0.1))
    return tf.reduce_mean(tf.reduce_sum(L, axis=1))

class MobileNetCapsNet:
    def __init__(self, input_shape=(224, 224, 3)):
        self.input_shape = input_shape
        self.model = self._build_model()
    
    def _build_model(self):
        base_model = tf.keras.applications.MobileNetV2(
            input_shape=self.input_shape,
            include_top=False,
            weights='imagenet'
        )
        base_model.trainable = False
        
        x = base_model.output
        x = layers.Conv2D(256, 3, activation='relu')(x)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Reshape((-1, 256))(x)
        
        x = CapsuleLayer(num_capsule=8, dim_capsule=16, routings=3)(x)
        x = CapsuleLayer(num_capsule=2, dim_capsule=32, routings=3)(x)
        outputs = Length()(x)
        
        return tf.keras.Model(inputs=base_model.input, outputs=outputs)
    
    def compile_model(self, learning_rate=0.001):
        """Compile the model with appropriate loss and optimizer"""
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        
        self.model.compile(
            optimizer=optimizer,
            loss=self.margin_loss,
            metrics=['accuracy']
        )
        
    @staticmethod
    def margin_loss(y_true, y_pred):
        """Margin loss for capsule network"""
        # Convert y_true to one-hot if it isn't already
        if len(K.int_shape(y_true)) == 1:
            y_true = tf.one_hot(tf.cast(y_true, 'int32'), 2)
            
        L = y_true * tf.square(tf.maximum(0., 0.9 - y_pred)) + \
            0.5 * (1 - y_true) * tf.square(tf.maximum(0., y_pred - 0.1))
        return tf.reduce_mean(tf.reduce_sum(L, axis=1))

In [4]:
from tensorflow.keras.models import load_model
# Load pre-trained global model
def load_model_from_file():
    return load_model(r"D:\Major Project\Rasp\old\drowsiness_model_teacher_our_final_10_epoch.keras", 
                      custom_objects={"CapsuleLayer": CapsuleLayer,"Length":Length,"margin_loss":margin_loss})#,

In [5]:
from pathlib import Path
INPUT_SHAPE = (224, 224, 3)
BATCH_SIZE = 16
TEST_DIR = r"D:\Major Project\Rasp\old\test"
# Evaluate the global model
def evaluate_model(model, dataset_dir):
# Define test image generator
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_gen = test_datagen.flow_from_directory(
        dataset_dir,
        target_size=INPUT_SHAPE[:2],
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )
    
    y_pred = np.argmax(model.predict(test_gen), axis=1)
    y_true = test_gen.classes
    
    print("\nTest Metrics:")
    print(f"Accuracy: {np.mean(y_true == y_pred):.4f}")
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=['Not Drowsy', 'Drowsy']))
# Server socket for federated learning
    

In [6]:
global_model = load_model_from_file()
evaluate_model(global_model,TEST_DIR)


Found 7045 images belonging to 2 classes.


D:\Major Project\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


441/441 ━━━━━━━━━━━━━━━━━━━━ 77s 172ms/step

Test Metrics:
Accuracy: 0.8453

Classification Report:
              precision    recall  f1-score   support

  Not Drowsy       0.85      0.87      0.86      3768
      Drowsy       0.84      0.82      0.83      3277

    accuracy                           0.85      7045
   macro avg       0.85      0.84      0.84      7045
weighted avg       0.85      0.85      0.85      7045



In [7]:
train_dir1 = r"D:\Major Project\Rasp\old\initial_train"
test_dir1 =  r"D:\Major Project\Rasp\old\test"

# Define ImageDataGenerators for training, validation, and testing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split for validation
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)

# Create training, validation, and test generators
train_gen = train_datagen.flow_from_directory(
    train_dir1,
    target_size=INPUT_SHAPE[:2],
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=False
)

val_gen = train_datagen.flow_from_directory(
    train_dir1,
    target_size=INPUT_SHAPE[:2],
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

test_gen = test_datagen.flow_from_directory(
    test_dir1,
    target_size=INPUT_SHAPE[:2],
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False  # Ensures label order consistency
)

Found 9019 images belonging to 2 classes.
Found 2253 images belonging to 2 classes.
Found 7045 images belonging to 2 classes.


In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, regularizers
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
class MobileNetStudent:
    def __init__(self, input_shape=(224, 224, 3), num_classes=2, learning_rate=3e-4):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.model = self._build_model()
        self._compile_model()

    def _build_model(self):
        base_model = tf.keras.applications.MobileNetV2(
            input_shape=self.input_shape,
            include_top=False,
            weights="imagenet",  # Use pre-trained weights
            alpha=0.35  # Reduced alpha to make it smaller
        )
        base_model.trainable = False  # Freeze the base model initially

        x = base_model.output
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dropout(0.6)(x)  # Increased dropout
        x = layers.Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.001))(x)  # Reduced units
        outputs = layers.Dense(self.num_classes, activation="softmax")(x)

        return Model(inputs=base_model.input, outputs=outputs)

    def _compile_model(self):
        optimizer = keras.optimizers.Adam(learning_rate=self.learning_rate)
        self.model.compile(
            optimizer=optimizer,
            loss="categorical_crossentropy",
            metrics=["CategoricalAccuracy"]
        )

    def get_callbacks(self):
        lr_scheduler = keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", factor=0.5, patience=2, verbose=1
        )
        early_stopping = keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=5, restore_best_weights=True
        )
        model_checkpoint = keras.callbacks.ModelCheckpoint(
            "best_model.h5", save_best_only=True, monitor="val_loss", mode="min"
        )
        return [lr_scheduler, early_stopping]


In [9]:
INPUT_SHAPE = (224, 224, 3)
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.001
TEST_SIZE = 0.2

In [10]:
student = MobileNetStudent()
model_student = student.model
model_student.compile(
    optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
    loss=CategoricalCrossentropy(from_logits=False),
    metrics=[CategoricalAccuracy()]) # Load once at the start
history2 = model_student.fit(
        train_gen,
        validation_data=val_gen,
        epochs=5,
        verbose=1)
model_student.save(r"D:\Major Project\Rasp\old\drowsiness_student_pre_model_our_final_5_epochs_wo_callbacks.keras")
evaluate_model(model_student,test_dir1)


D:\Major Project\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
564/564 ━━━━━━━━━━━━━━━━━━━━ 148s 255ms/step - categorical_accuracy: 0.5325 - loss: 0.8841 - val_categorical_accuracy: 0.5344 - val_loss: 0.7626
Epoch 2/5
564/564 ━━━━━━━━━━━━━━━━━━━━ 142s 251ms/step - categorical_accuracy: 0.5785 - loss: 0.7462 - val_categorical_accuracy: 0.5766 - val_loss: 0.7249
Epoch 3/5
564/564 ━━━━━━━━━━━━━━━━━━━━ 141s 251ms/step - categorical_accuracy: 0.6507 - loss: 0.6641 - val_categorical_accuracy: 0.5269 - val_loss: 0.7890
Epoch 4/5
564/564 ━━━━━━━━━━━━━━━━━━━━ 140s 248ms/step - categorical_accuracy: 0.6540 - loss: 0.6443 - val_categorical_accuracy: 0.6507 - val_loss: 0.7498
Epoch 5/5
564/564 ━━━━━━━━━━━━━━━━━━━━ 142s 252ms/step - categorical_accuracy: 0.7077 - loss: 0.5983 - val_categorical_accuracy: 0.5850 - val_loss: 0.7246
Found 7045 images belonging to 2 classes.
221/221 ━━━━━━━━━━━━━━━━━━━━ 44s 194ms/step

Test Metrics:
Accuracy: 0.7005

Classification Report:
              precision    recall  f1-score   support

  Not Drowsy       0.67     

In [11]:
import tensorflow as tf
from tensorflow import keras
import os

class ServerDistiller(keras.Model):
    def __init__(self, teacher, student, temp=3.0, alpha=0.1, grad_clip=1.0):
        super().__init__()
        self.teacher = teacher
        self.student = student
        self.temp = temp
        self.alpha = alpha
        self.grad_clip = grad_clip
        self.teacher.trainable = False  # Freeze the teacher model

        # Metrics
        self.total_loss_metric = keras.metrics.Mean(name="total_loss")
        self.student_loss_metric = keras.metrics.Mean(name="student_loss")
        self.distill_loss_metric = keras.metrics.Mean(name="distill_loss")
        self.acc_metric = keras.metrics.CategoricalAccuracy(name="accuracy")

    def compile(self, optimizer, **kwargs):
        """Properly compiles the model with loss validation bypass."""
        kwargs.pop('loss', None)  # Prevent Keras validation issues
        super().compile(optimizer=optimizer, loss=self._dummy_loss, **kwargs)

        # Define actual loss functions
        self.student_loss_fn = keras.losses.CategoricalCrossentropy(from_logits=False)  
        self.distill_loss_fn = keras.losses.KLDivergence()

    def _dummy_loss(self, y_true, y_pred):
        """Dummy loss function to bypass Keras validation checks."""
        return 0.0

    def train_step(self, data):
        """Custom training step for knowledge distillation."""
        x, y = data

        with tf.GradientTape() as tape:
            teacher_logits = self.teacher(x, training=False)  # Teacher inference mode
            student_probs = self.student(x, training=True)  # Student training mode

            # Compute student loss (cross-entropy with true labels)
            student_loss = self.student_loss_fn(y, student_probs)

            # Compute distillation loss (teacher-student KL divergence)
            teacher_probs = tf.nn.softmax(teacher_logits / self.temp, axis=1)
            distill_loss = (self.temp ** 2) * self.distill_loss_fn(teacher_probs, student_probs)  # Scale KL divergence

            # Total loss: weighted sum of both
            total_loss = self.alpha * student_loss + (1 - self.alpha) * distill_loss

        # Compute gradients & apply clipping
        gradients = tape.gradient(total_loss, self.student.trainable_variables)
        gradients = [tf.clip_by_norm(g, self.grad_clip) for g in gradients]
        self.optimizer.apply_gradients(zip(gradients, self.student.trainable_variables))

        # Update metrics
        self.total_loss_metric.update_state(total_loss)
        self.student_loss_metric.update_state(student_loss)
        self.distill_loss_metric.update_state(distill_loss)
        self.acc_metric.update_state(y, student_probs)

        return {m.name: m.result() for m in self.metrics}

    @property
    def metrics(self):
        """Returns list of tracked metrics."""
        return [
            self.total_loss_metric,
            self.student_loss_metric,
            self.distill_loss_metric,
            self.acc_metric
        ]

    def call(self, inputs, training=False):
        """Forward pass using student model."""
        return self.student(inputs, training=training)

def plot_kd_metrics(distiller, title_prefix="KD Training"):
    epochs = range(1, len(distiller.history_total_loss) + 1)

    plt.figure(figsize=(12, 5))

    # Losses
    plt.subplot(1, 2, 1)
    plt.plot(epochs, distiller.history_total_loss, label="Total Loss")
    plt.plot(epochs, distiller.history_student_loss, label="Student Loss (CE)")
    plt.plot(epochs, distiller.history_distill_loss, label="Distillation Loss (KL)")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title(f"{title_prefix} - Training Losses")
    plt.legend()

    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, distiller.history_accuracy, label="Train Accuracy", color="green")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title(f"{title_prefix} - Train Accuracy")
    plt.legend()

    plt.tight_layout()
    plt.show()



In [12]:
teacher_model=global_model
distiller = ServerDistiller(teacher_model, model_student, temp=3.0, alpha=0.1)
distiller.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[distiller.acc_metric]
)
distiller.fit(train_gen, epochs=10, verbose=1)
distiller.student.save(r"D:\Major Project\Rasp\old\drowsiness_student_post_model_5_epochs_our_final_wo_callbacks.keras")
print("Evaluation of distilled student")
evaluate_model(distiller.student, test_dir1)


Epoch 1/10
564/564 ━━━━━━━━━━━━━━━━━━━━ 194s 331ms/step - accuracy: 0.6047 - distill_loss: 0.2769 - student_loss: 0.6368 - total_loss: 0.3129
Epoch 2/10
564/564 ━━━━━━━━━━━━━━━━━━━━ 186s 329ms/step - accuracy: 0.6082 - distill_loss: 0.0128 - student_loss: 0.6821 - total_loss: 0.0797
Epoch 3/10
564/564 ━━━━━━━━━━━━━━━━━━━━ 187s 332ms/step - accuracy: 0.6414 - distill_loss: 0.0114 - student_loss: 0.6807 - total_loss: 0.0783
Epoch 4/10
564/564 ━━━━━━━━━━━━━━━━━━━━ 186s 329ms/step - accuracy: 0.6422 - distill_loss: 0.0106 - student_loss: 0.6794 - total_loss: 0.0775
Epoch 5/10
564/564 ━━━━━━━━━━━━━━━━━━━━ 186s 330ms/step - accuracy: 0.6515 - distill_loss: 0.0107 - student_loss: 0.6764 - total_loss: 0.0773
Epoch 6/10
564/564 ━━━━━━━━━━━━━━━━━━━━ 187s 332ms/step - accuracy: 0.6795 - distill_loss: 0.0103 - student_loss: 0.6767 - total_loss: 0.0769
Epoch 7/10
564/564 ━━━━━━━━━━━━━━━━━━━━ 186s 329ms/step - accuracy: 0.7094 - distill_loss: 0.0107 - student_loss: 0.6752 - total_loss: 0.0772
Epoch 

D:\Major Project\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


221/221 ━━━━━━━━━━━━━━━━━━━━ 42s 191ms/step

Test Metrics:
Accuracy: 0.7012

Classification Report:
              precision    recall  f1-score   support

  Not Drowsy       0.83      0.55      0.66      3768
      Drowsy       0.63      0.87      0.73      3277

    accuracy                           0.70      7045
   macro avg       0.73      0.71      0.70      7045
weighted avg       0.74      0.70      0.70      7045

